# 风管模型第四次作业

## 第一题

### LN 模型

In [1]:
import numpy as np
import pandas as pd
from arch import arch_model
from scipy.stats import chi2
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression


df = pd.read_excel("./lib/创业板指.xlsx", index_col=2,skipfooter=2).sort_index().drop(columns=["代码","名称"])
df = df.apply(
    lambda x: x.str.replace(",", "").astype(float) if x.dtype == "object" else x
)
df.index = pd.to_datetime(df.index)
result = {}
df["weeknum"] = df.index.map(lambda x: str(x.year) + "-" + str(x.isocalendar().week))
df = df.drop_duplicates(subset="weeknum")
# log = df["收盘价"].apply(np.log)
log = (df["收盘价(元)"] / df["收盘价(元)"].shift(1)).dropna().apply(np.log)
mu = log.mean()
sigma = log.std()
likelyhood_ln = log.apply(
    lambda x: -np.log(sigma) - np.log(2 * np.pi) / 2 - (x - mu) ** 2 / (2 * sigma**2)
).sum()
k, logl = 2, likelyhood_ln
result["LN"] = {
    "k":k,
    "log(L)": logl,
    "AIC": logl - k,
    "SBC": logl - k * np.log(log.shape[0]) / 2,
}
pd.DataFrame(result).T

,AIC,SBC,k,log(L)
LN,1022.312383,1017.890761,2.0,1024.312383


### AR(1) 模型

In [2]:
model = ARIMA(log.values, order=(1, 0, 0))
AR1 = model.fit()
k, logl = 3, AR1.llf
result["AR(1)"] = {
    "k":k,
    "log(L)": logl,
    "AIC": logl - k,
    "SBC": logl - k * np.log(log.shape[0]) / 2,
}
print(AR1.summary())
pd.DataFrame(result).T

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  615
Model:                 ARIMA(1, 0, 0)   Log Likelihood                1024.615
Date:                Tue, 07 Jun 2022   AIC                          -2043.230
Time:                        20:32:07   BIC                          -2029.965
Sample:                             0   HQIC                         -2038.072
                                - 615                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0015      0.002      0.800      0.424      -0.002       0.005
ar.L1         -0.0314      0.035     -0.908      0.364      -0.099       0.036
sigma2         0.0021   8.75e-05     23.886      0.0

,k,log(L),AIC,SBC
LN,2.0,1024.312383,1022.312383,1017.890761
AR(1),3.0,1024.614985,1021.614985,1014.982551


### ARCH 模型

In [3]:
arch = arch_model(log, p=1, q=0, rescale=False).fit(update_freq=0)
k, logl = 3, arch.loglikelihood
print(arch.summary())
result["ARCH"] = {
    "k":k,
    "log(L)": logl,
    "AIC": logl - k,
    "SBC": logl - k * np.log(log.shape[0]) / 2,
}
pd.DataFrame(result).T

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1001.7471367001739
            Iterations: 6
            Function evaluations: 14
            Gradient evaluations: 2
                      Constant Mean - ARCH Model Results                      
Dep. Variable:                 收盘价(元)   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                       ARCH   Log-Likelihood:                1001.75
Distribution:                  Normal   AIC:                          -1997.49
Method:            Maximum Likelihood   BIC:                          -1984.23
                                        No. Observations:                  615
Date:                Tue, Jun 07 2022   Df Residuals:                      614
Time:                        20:32:07   Df Model:                            1
                                  Mean Model                                 
  

,k,log(L),AIC,SBC
LN,2.0,1024.312383,1022.312383,1017.890761
AR(1),3.0,1024.614985,1021.614985,1014.982551
ARCH,3.0,1001.747137,998.747137,992.114703


### GARCH 模型

In [4]:
garch = arch_model(log, p=1, q=1, rescale=False).fit(update_freq=0)
k, logl = 4, garch.loglikelihood
print(garch.summary())
result["GARCH"] = {
    "k":k,
    "log(L)": logl,
    "AIC": logl - k,
    "SBC": logl - k * np.log(log.shape[0]) / 2,
}
pd.DataFrame(result).T

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1059.9898674281862
            Iterations: 7
            Function evaluations: 27
            Gradient evaluations: 3
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                 收盘价(元)   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                1059.99
Distribution:                  Normal   AIC:                          -2111.98
Method:            Maximum Likelihood   BIC:                          -2094.29
                                        No. Observations:                  615
Date:                Tue, Jun 07 2022   Df Residuals:                      614
Time:                        20:32:08   Df Model:                            1
                                  Mean Model                                 
  

,k,log(L),AIC,SBC
LN,2.0,1024.312383,1022.312383,1017.890761
AR(1),3.0,1024.614985,1021.614985,1014.982551
ARCH,3.0,1001.747137,998.747137,992.114703
GARCH,4.0,1059.989867,1055.989867,1047.146623


### RSLN-2

[reference](https://www.statsmodels.org/dev/generated/statsmodels.tsa.regime_switching.markov_regression.MarkovRegression.html)

In [5]:
rsln = MarkovRegression(log.values, 2, switching_variance=True).fit() # 异方差
k, logl = 6, rsln.llf
print(rsln.summary())
result["RSLN-2"] = {
    "k":k,
    "log(L)": logl,
    "AIC": logl - k,
    "SBC": logl - k * np.log(log.shape[0]) / 2,
    "LRT": None,
}
pd.DataFrame(result).T

                        Markov Switching Model Results                        
Dep. Variable:                      y   No. Observations:                  615
Model:               MarkovRegression   Log Likelihood                1052.556
Date:                Tue, 07 Jun 2022   AIC                          -2093.113
Time:                        20:32:11   BIC                          -2066.583
Sample:                             0   HQIC                         -2082.797
                                - 615                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0012      0.002      0.587      0.557      -0.003       0.005
sigma2         0.0012      0.000      5.641      0.0

,k,log(L),AIC,SBC,LRT
LN,2.0,1024.312383,1022.312383,1017.890761,NaN
AR(1),3.0,1024.614985,1021.614985,1014.982551,NaN
ARCH,3.0,1001.747137,998.747137,992.114703,NaN
GARCH,4.0,1059.989867,1055.989867,1047.146623,NaN
RSLN-2,6.0,1052.556303,1046.556303,1033.291436,NaN


### Likelihood Ratio Test

In [6]:
basemodel = "LN"
for model in result:
    if model == basemodel:
        continue
    result[model]["LRT"] = chi2.sf(result[model]["log(L)"] - result[basemodel]["log(L)"], result[model]["k"] - result[basemodel]["k"])
pd.DataFrame(result).T

,k,log(L),AIC,SBC,LRT
LN,2.0,1024.312383,1022.312383,1017.890761,NaN
AR(1),3.0,1024.614985,1021.614985,1014.982551,5.822561e-01
ARCH,3.0,1001.747137,998.747137,992.114703,1.000000e+00
GARCH,4.0,1059.989867,1055.989867,1047.146623,1.789504e-08
RSLN-2,6.0,1052.556303,1046.556303,1033.291436,1.113061e-05


## 第二题

In [7]:
monte_carlo = {}
np.random.RandomState(42)
cases = 10000
for case in range(cases):
    if (case*10) % cases == 0:
        print(case/cases*100, "%")
    sim_model = arch_model(None, p=1, q=1)
    simulation = sim_model.simulate(garch.params, 26)
    monte_carlo[case] = np.prod(1 + simulation["data"])

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %


In [8]:
monte_carlo = pd.Series(monte_carlo)
var = pd.Series(monte_carlo).quantile([0.01, 0.05, 0.1])
es = var.apply(lambda x: monte_carlo[monte_carlo < x].mean())
pd.DataFrame({"es": 1 - es, "var": var - 1})

,es,var
0.01,0.590862,-0.507615
0.05,0.437099,-0.341071
0.10,0.366649,-0.261979
